## **PUBMED Dataset Preprocessing**

Imports

In [1]:
from src.training_utils import *
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

print(transformers.__version__)

/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: difra00 (deepl_wizards). Use `wandb login --relogin` to force relogin


4.30.2


Load the Pubmed dataset which was downloaded to the local directory 

In [2]:
pubmed_dataset = load_dataset("ccdv/pubmed-summarization")
print(pubmed_dataset)

No config specified, defaulting to: pubmed-summarization/section
Found cached dataset pubmed-summarization (/home/peppe/.cache/huggingface/datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b)
100%|██████████| 3/3 [00:00<00:00, 987.75it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6658
    })
})


Load and Visualize the Dataset, cleaning two examples in the validation

In [2]:
import json
from tqdm import tqdm

def load_corpus(fname, is_training):
    corpus = []
    with open(fname, "r") as f:
        lines = f.readlines()  # Read all lines into a list
        for i, line in tqdm(enumerate(lines), total=len(lines)):
            data = json.loads(line)
            if len(data["text"]) == 0 or len(data["summary"]) == 0:
                print(i)
                continue
            if is_training:
                if len(data["indices"]) == 0 or len(data["score"]) == 0:
                    continue

            corpus.append(data)
    return corpus

val = load_corpus("src/data/PubMed/val_PUBMED.jsonl", False)
train = load_corpus("src/data/PubMed/train_PUBMED_labelled.jsonl", True)


100%|██████████| 6633/6633 [00:00<00:00, 24736.74it/s]

2320
4923



100%|██████████| 116937/116937 [00:06<00:00, 18319.82it/s]


In [4]:
pubmed_dataset['validation'][2320]

{'article': ' \n  \n  \n  \n ',
 'abstract': "according to the central limit theorem , the means of a random sample of size , n , from a population with mean ,  , and variance , 2 , distribute normally with mean ,  , and variance , 2n . using the central limit theorem , \n a variety of parametric tests have been developed under assumptions about the parameters that determine the population probability distribution . \n compared to non - parametric tests , which do not require any assumptions about the population probability distribution , parametric tests produce more accurate and precise estimates with higher statistical powers . \n however , many medical researchers use parametric tests to present their data without knowledge of the contribution of the central limit theorem to the development of such tests . \n thus , this review presents the basic concepts of the central limit theorem and its role in binomial distributions and the student 's t - test , and provides an example of the

In [5]:
pubmed_dataset['validation'][4923]

{'article': ' \n  \n  \n  \n  \n ',
 'abstract': 'highlightswe present a rare case of a congenital splenic cyst that corresponds to less than 10% of non - parasitic splenic cyst.we describe our laparoscopic approach management of a partial splenectomywe review the literature on the different treatment options for npsc based on recurrence and success rates .'}

## **Clean Dataset Class**

In [6]:
from torch.utils.data import Dataset

class Clean_Dataset(Dataset):
    # This is used to clean the noisy sample from PubMed train
    def __init__(self, abs_dataset, ext_dataset, val = False):
        self.abs_dataset = abs_dataset
        self.ext_dataset = ext_dataset
        self.abs_dataset, self.ext_dataset = self.preprocess(val)


    def __len__(self):
        return len(self.ext_dataset)
    
    def __getitem__(self, idx):

        return self.abs_dataset[idx], self.ext_dataset[idx]
    
    def compare(self,s1,s2, red = True):
        distance = (s1[:len(s2)+10].strip()).startswith(s2.strip())#Levenshtein.distance(s1[:100], s2[:100])
        if not distance:
            if red:
                print(f"\nEXT: \n", s2)
                print(f"\nABS: \n", s1[:len(s2)+10])
            return False
        else:
            return True
            
    
    def preprocess(self, val = False):
        new_dataset_abs = []
        new_dataset = []
        new_ext = []
        count = 0
        for idx in tqdm(range(len(self.abs_dataset))):

            if self.abs_dataset[idx]['article'] != "":
                if not val:
                    new_dataset_abs.append({"article": self.abs_dataset[idx]['article'], 'abstract': self.abs_dataset[idx]['abstract']})
                elif val and idx not in [4923, 2320]:
                    new_dataset_abs.append({"article": self.abs_dataset[idx]['article'], 'abstract': self.abs_dataset[idx]['abstract']})

            else:
                count+=1
        if not val:
            for idx in range(len(self.ext_dataset)):
                if self.compare(new_dataset_abs[idx]['article'],self.ext_dataset[idx]['text'][0]):
                    new_dataset.append({"article": new_dataset_abs[idx]['article'], 'abstract': new_dataset_abs[idx]['abstract']})
                    new_ext.append(self.ext_dataset[idx])
                    continue
                    
                else:
                    count+=1
                    for idx2 in range(1, len(new_dataset_abs[idx:])):
                        if self.compare(new_dataset_abs[idx2+idx]['article'],self.ext_dataset[idx]['text'][0], red = False):
                            new_dataset.append({"article": new_dataset_abs[idx2+idx]['article'], 'abstract': new_dataset_abs[idx2+idx]['abstract']})
                            new_ext.append(self.ext_dataset[idx])
                            break
        else:
            new_dataset = new_dataset_abs
            new_ext = self.ext_dataset

        return new_dataset, new_ext
        

Run only once

In [7]:
#train_new = Clean_Dataset(pubmed_dataset['train'], train, val = False)
#val_new = Clean_Dataset(pubmed_dataset['validation'], val, val = True)
#print(len(train_new))
#print(len(val_new))

In [8]:
#serialized_data_val = [sample for sample in val_new]
#serialized_data_train = [sample for sample in val_new]

In [9]:
#with open("src/data/PubMed/Train_ExtAbs_PUBMED.json", "w") as f:
#    json.dump(serialized_data_train, f)

# with open("src/data/PubMed/Val_ExtAbs_PUBMED.json", "w") as f:
#    json.dump(serialized_data_val, f)

Load the saved dataset by opening the .json file

In [3]:
with open("src/data/PubMed/Train_ExtAbs_PUBMED.json") as f:
        training_corpus = json.load(f)

In [4]:
len(training_corpus) # len is 113754 # Removed the sentences that were not in the abstractive, and grouped them correctly.

113754

In [11]:
#Visualize an example
training_corpus[0][1]

{'text': ['a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries .',
  'in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively .',
  'the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% .',
  'anthropometric study of elementary school students in shiraz revealed that 16% of them suffer from malnutrition and low body weight .',
  'snack should have 300 - 400 kcal energy and could provide 5 - 10 g of protein / day . nowadays , school nutrition programs are running as the national programs , world - wide . national school lunch program in the united states',
  'there are also some reports regarding school feeding programs in 

In [6]:
with open("src/data/PubMed/Val_ExtAbs_PUBMED.json") as f:
        validation_corpus = json.load(f)
len(validation_corpus) # len is 6631 # removed the same sentences of the original

6631

In [13]:
#Visualize an example
validation_corpus[0][1]

{'summary': ['background and aim : there is lack of substantial indian data on venous thromboembolism ( vte ) .',
  'the aim of this study was to provide real - world information on patient characteristics , management strategies , clinical outcomes , and temporal trends in vte.subjects and methods : multicentre retrospective registry involving 549 medical records of patients with confirmed diagnosis of vte ( deep vein thrombosis [ dvt ] confirmed by doppler ultrasonography ; pulmonary embolism [ pe ] by computed tomography , pulmonary angiography and/or v / q scan ) from 2006 to 2010 at three indian tertiary care hospitals.results:acute dvt without pe , acute dvt with pe , and pe alone were reported in 64% ( 352/549 ) , 23% ( 124/549 ) , and 13% ( 73/549 ) patients , respectively .',
  'mean age was 47 ( 16 ) years , and 70% were males .',
  'h / o dvt ( 34% ) , surgery including orthopedic surgery ( 28% ) , trauma ( 16% ) , and immobilization > 3 days ( 14% ) were the most common ris